In [59]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import csv
from scipy import sparse
import numpy as np
import sklearn.manifold
import sklearn.datasets
from sklearn.decomposition import PCA
import MulticoreTSNE
from sklearn.utils import shuffle

In [60]:
twenty_train = fetch_20newsgroups(shuffle=True, random_state=42)

In [61]:
print (twenty_train.data[1])
print (twenty_train.target_names[1])

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>

comp.graphics


In [62]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
vocab = count_vect.vocabulary_ 



In [63]:
print (type(vocab))


<class 'dict'>


In [64]:
print (X_train_counts.get_shape())

(11314, 130107)


In [65]:
sparse.save_npz('data_sparse.npz', X_train_counts)


In [66]:
def save_sparse_matrix(m,filename, header=''):
    thefile = open(filename, 'w')
    thefile.write('FIRST_KEY, SECOND_KEY, OCCURENCES \n')
    nonZeros = np.array(m.nonzero()) 
    for entry in range(nonZeros.shape[1]):
        thefile.write("%s,%s,%s\n" % (nonZeros[0, entry], nonZeros[1, entry], m[nonZeros[0, entry], nonZeros[1, entry]]))

In [69]:
save_sparse_matrix(X_train_counts, "20groups_bag_of_words.csv")

In [68]:
with open('20groups_word_mapping.csv', 'w') as f:  
    w = csv.writer(f)
    f.write('WORD,KEY \n')
    w.writerows(vocab.items())

In [ ]:
digits = sklearn.datasets.load_digits()
orig_X = digits.data
orig_Y = digits.target

every_nth_element = 1 #defult 1 == every element
X = orig_X[0::every_nth_element]
y = orig_Y[0::every_nth_element]

print("Computing t-SNE embedding")
tsne = sklearn.manifold.TSNE(n_components=3, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X)

print(X_tsne)



Computing t-SNE embedding


In [ ]:
print (X_tsne.shape)
print (y_reshaped.shape)
y_reshaped = y.reshape(1797, 1)

merged_X_Y = np.concatenate((X_tsne, y_reshaped), axis=1)

In [ ]:
numpy.savetxt("mnist_small_tsne.csv", merged_X_Y, fmt='%.5e',  delimiter=",", 
              header='X,Y,Z,label', comments='') 

In [55]:
def reduce_mnist_and_save(number_of_digits):
    mnist = sklearn.datasets.fetch_mldata('MNIST original')
    mnist_X, mnist_Y = shuffle(mnist.data, mnist.target)
    full_mnist_data = mnist_X[:number_of_digits]
    full_mnist_labels = mnist_Y[:number_of_digits]
    pca = PCA(n_components=30)
    full_mnist_after_pca = pca.fit_transform(full_mnist_data)
    tsne = MulticoreTSNE.MulticoreTSNE(n_components=3, random_state=0, n_jobs=8)
    full_mnist_after_tsne = tsne.fit_transform(full_mnist_after_pca)
    full_mnist_labels = full_mnist_labels.reshape(number_of_digits, 1)
    merged_full_mnist = np.concatenate((full_mnist_after_tsne, full_mnist_labels), axis=1)
    file_name = "mnist_" + str(number_of_digits) + ".csv"
    np.savetxt(file_name, merged_full_mnist, fmt='%.5e',  delimiter=",", 
              header='X,Y,Z,label', comments='')
    

In [56]:
reduce_mnist_and_save(70000)